In [ ]:
from result_helpers import *
import os

In [ ]:
naive_dir = "_tmp_loghi/page"

naive_data = []
for root, dirs, files in os.walk(naive_dir):
    for file in files:
        if file.endswith(".xml"):
            xml_file = os.path.join(root, file)
            naive_data.append(loghi_2_csv(xml_file))

ground_truth = []
ground_truth_dir = "input_jpg_xml"
for root, dirs, files in os.walk(ground_truth_dir):
    for file in files:
        if file.endswith(".xml"):
            xml_file = os.path.join(root, file)
            ground_truth += [i for i in parse_ground_truth(xml_file)]


import pickle

better_data = pickle.load(open("all_results.pkl", "rb"))

naive_data = [i for i in naive_data if i]

print("Naive data: ", len(naive_data))
print("Ground truth: ", len(ground_truth))
print("Better data: ", len(better_data))

In [144]:
def print_stats(matches):
    all_cer = []
    all_iou = []
    all_iou_best = []
    all_cer_best = []
    for i, j in matches:
        r1 = i.compare_cer(j)
        if r1 is not None:
            all_cer.append(r1)

        r2 = i.compare_iou(j)
        if r2 is not None:
            all_iou.append(r2)

        r3 = i.comapre_best_iou(j, th=0.25)
        if r3 is not None:
            all_iou_best.append(r3)

        r4 = i.compare_best_cer(j)
        if r4 is not None:
            all_cer_best.append(r4)


    print("Average CER: ", sum(all_cer) / len(all_cer), len(all_cer))
    print("Average IOU: ", sum(all_iou) / len(all_iou), len(all_iou))
    print("Average IOU best: ", sum(all_iou_best) / len(all_iou_best), len(all_iou_best))
    print("Average CER best: ", sum(all_cer_best) / len(all_cer_best), len(all_cer_best))

def compare_subtotals(matches):
    count = 0
    for data1, data2 in matches:
        print(data1.name)

        st1 = data1.subtotals()
        st2 = data2.subtotals()
        for k2 in st2:
            # Find the closest match in st1
            best = None
            best_diff = 0
            print(k2.data, k2.region)
            for k1 in st1:
                k1.region.r = [(int(a),int(b)) for a,b in k1.region.r]
                diff = k1.region.iou(k2.region)                
                print(diff, k1.data, end=" ")
                print(k1.region)
                if diff > best_diff:
                    best_diff = diff
                    best = k1

        print(best, best_diff)
        break

    return count / len(matches)


In [ ]:
matches1 = []

for item in better_data:
    better_name = item.name
    for gt in ground_truth:
        ground_truth_name = gt.name.split("/")[-1].removesuffix(".csv")
        if better_name == ground_truth_name:
            matches1.append((item, gt))
            break

print_stats(matches1)
print(compare_subtotals(matches1))
matches2 = []

for item in naive_data:
    naive_data_name = item.name.split("/")[-1].removesuffix("-naive.csv")
    for gt in ground_truth:
        ground_truth_name = gt.name.split("/")[-1].removesuffix(".csv")
        if naive_data_name == ground_truth_name:
            matches2.append((item, gt))
            break
print_stats(matches2)